In [25]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
from matplotlib import rc

# 한글 폰트 사용 위함
rc('font', family='Malgun Gothic')

from sklearn.preprocessing import LabelEncoder
import xgboost as xgb


In [26]:
# train, test 데이터를 불러옵니다.
train = pd.read_csv('./train.csv', index_col='ID')
test = pd.read_csv('./test.csv', index_col='ID')

In [27]:
train

,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
ID,,,,,,,,,,
TRAIN_0000,P사,TayGTS,Nearly New,86.077,AWD,13642,0,No,2,159.66
TRAIN_0001,K사,Niro,Nearly New,56.000,FWD,10199,6,No,0,28.01
TRAIN_0002,A사,eT,Brand New,91.200,AWD,2361,7,No,0,66.27
TRAIN_0003,A사,RSeTGT,Nearly New,NaN,AWD,21683,3,No,0,99.16
TRAIN_0004,B사,i5,Pre-Owned,61.018,AWD,178205,1,No,0,62.02
...,...,...,...,...,...,...,...,...,...,...
TRAIN_7492,H사,ION5,Brand New,NaN,AWD,3773,10,No,0,35.95
TRAIN_7493,B사,i3,Pre-Owned,46.000,RWD,135411,2,No,0,23.40
TRAIN_7494,P사,TayCT,Brand New,NaN,AWD,1363,2,No,0,120.00


In [28]:
from sklearn.preprocessing import LabelEncoder

# 5개의 범주형 컬럼 각각 라벨 인코딩
categorical_columns = ['모델', '제조사', '구동방식', '차량상태', '사고이력']
for col in categorical_columns:
    encoder = LabelEncoder()
    train[col] = encoder.fit_transform(train[col])
print(train.head())

            제조사  모델  차량상태   배터리용량  구동방식  주행거리(km)  보증기간(년)  사고이력  연식(년)  \
ID                                                                        
TRAIN_0000    4  16     1  86.077     0     13642        0     0      2   
TRAIN_0001    3  10     1  56.000     1     10199        6     0      0   
TRAIN_0002    0  17     0  91.200     0      2361        7     0      0   
TRAIN_0003    0  12     1     NaN     0     21683        3     0      0   
TRAIN_0004    1  19     2  61.018     0    178205        1     0      0   

            가격(백만원)  
ID                   
TRAIN_0000   159.66  
TRAIN_0001    28.01  
TRAIN_0002    66.27  
TRAIN_0003    99.16  
TRAIN_0004    62.02  


In [33]:
# 배터리 용량이 있는 데이터 (학습 데이터)
train_with_target = train[train['배터리용량'].notna()]
X_train = train_with_target.drop(columns=['배터리용량','가격(백만원)'])
y_train = train_with_target['배터리용량']

# 배터리 용량이 없는 데이터 (테스트 데이터)
test_without_target = train[train['배터리용량'].isna()]
X_test = test_without_target.drop(columns=['배터리용량','가격(백만원)'])
#y_test = test_without_target['배터리용량']  # 실제 값 (만약 존재한다면)

In [38]:
print(X_train.head())
print(y_train.head())
print(X_test.head())
print(y_test.head())

            제조사  모델  차량상태  구동방식  주행거리(km)  보증기간(년)  사고이력  연식(년)
ID                                                             
TRAIN_0000    4  16     1     0     13642        0     0      2
TRAIN_0001    3  10     1     1     10199        6     0      0
TRAIN_0002    0  17     0     0      2361        7     0      0
TRAIN_0004    1  19     2     0    178205        1     0      0
TRAIN_0005    2   3     2     0    103100        3     0      0
ID
TRAIN_0000    86.077
TRAIN_0001    56.000
TRAIN_0002    91.200
TRAIN_0004    61.018
TRAIN_0005    58.162
Name: 배터리용량, dtype: float64
            제조사  모델  차량상태  구동방식  주행거리(km)  보증기간(년)  사고이력  연식(년)
ID                                                             
TRAIN_0003    0  12     1     0     21683        3     0      0
TRAIN_0006    5   7     1     0     19395        3     0      0
TRAIN_0008    5   9     0     0      2226        8     0      0
TRAIN_0009    0  11     0     0      3683        7     0      0
TRAIN_0011    6   1     0     0 

In [34]:
X_train

,제조사,모델,차량상태,구동방식,주행거리(km),보증기간(년),사고이력,연식(년)
ID,,,,,,,,
TRAIN_0000,4,16,1,0,13642,0,0,2
TRAIN_0001,3,10,1,1,10199,6,0,0
TRAIN_0002,0,17,0,0,2361,7,0,0
TRAIN_0004,1,19,2,0,178205,1,0,0
TRAIN_0005,2,3,2,0,103100,3,0,0
...,...,...,...,...,...,...,...,...
TRAIN_7486,0,17,0,0,602,7,0,0
TRAIN_7487,2,4,1,1,29028,3,0,1
TRAIN_7493,1,18,2,2,135411,2,0,0


In [35]:
X_test

,제조사,모델,차량상태,구동방식,주행거리(km),보증기간(년),사고이력,연식(년)
ID,,,,,,,,
TRAIN_0003,0,12,1,0,21683,3,0,0
TRAIN_0006,5,7,1,0,19395,3,0,0
TRAIN_0008,5,9,0,0,2226,8,0,0
TRAIN_0009,0,11,0,0,3683,7,0,0
TRAIN_0011,6,1,0,0,1131,8,0,0
...,...,...,...,...,...,...,...,...
TRAIN_7489,2,2,0,0,8871,9,0,0
TRAIN_7490,0,11,0,0,5794,7,0,0
TRAIN_7491,3,13,0,1,5966,10,0,0


In [36]:
# 모델 학습
xgb_model = xgb.XGBRegressor(random_state=42)
xgb_model.fit(X_train, y_train)

# 예측
y_pred = xgb_model.predict(X_test)


In [40]:
print(y_pred)

[77.92845  81.90777  90.06781  ... 89.946045 90.00233  90.332466]


In [37]:
# 성능 평가 (예: RMSE 계산)
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(X_test, y_pred))
print(f"RMSE: {rmse:.4f}")

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').